## Proceso ETL - Database de películas  
Se explican los pasos de ejecución de Extracción, Carga y limpieza de datos para exponer un dataset adecuado para las consultas necesarias

### 1) Importación de librerías a utilizar en el procedimiento

In [10]:
#Librerías
import pandas as pd 
import numpy as np
import pymysql #Para conexión con la base datos

### 2) Carga de datasets y exploración de datos con pandas  (EDA) 
* Se utiliza python para verificar realizar un anális EDA (Exploratory Data Analysis) de los datos (verificar duplicados, nulos, etc.)
* Adicionalmente, se lee el archivo _netflix_titles.json_ con pandas y luego se guarda como archivo `.csv` para cargarlo en MySQL

In [6]:
df_amazon = pd.read_csv('../Data/amazon_prime_titles.csv')
df_disney = pd.read_csv('../Data/disney_plus_titles.csv')
df_hulu = pd.read_csv('../Data/hulu_titles.csv')
df_netflix = pd.read_json('../Data/netflix_titles.json')

In [7]:
#Diccionario de Dataframes para facilitar las consultas
dict_df = {
            'df_amazon':df_amazon, 
            'df_disney':df_disney,
            'df_hulu':df_hulu,
            'df_netflix':df_netflix
            }

* Head de los Datasets

In [51]:
df_amazon.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...
3,s4,Movie,Pink: Staying True,Sonia Anderson,"Interviews with: Pink, Adele, Beyoncé, Britney...",United States,"March 30, 2021",2014,NaN,69 min,Documentary,"Pink breaks the mold once again, bringing her ..."
4,s5,Movie,Monster Maker,Giles Foster,"Harry Dean Stanton, Kieran O'Brien, George Cos...",United Kingdom,"March 30, 2021",1989,NaN,45 min,"Drama, Fantasy",Teenage Matt Banting wants to work with a famo...


In [52]:
df_disney.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Duck the Halls: A Mickey Mouse Christmas Special,"Alonso Ramirez Ramos, Dave Wasson","Chris Diamantopoulos, Tony Anselmo, Tress MacN...",NaN,"November 26, 2021",2016,TV-G,23 min,"Animation, Family",Join Mickey and the gang as they duck the halls!
1,s2,Movie,Ernest Saves Christmas,John Cherry,"Jim Varney, Noelle Parker, Douglas Seale",NaN,"November 26, 2021",1988,PG,91 min,Comedy,Santa Claus passes his magic bag to a new St. ...
2,s3,Movie,Ice Age: A Mammoth Christmas,Karen Disher,"Raymond Albert Romano, John Leguizamo, Denis L...",United States,"November 26, 2021",2011,TV-G,23 min,"Animation, Comedy, Family",Sid the Sloth is on Santa's naughty list.
3,s4,Movie,The Queen Family Singalong,Hamish Hamilton,"Darren Criss, Adam Lambert, Derek Hough, Alexa...",NaN,"November 26, 2021",2021,TV-PG,41 min,Musical,"This is real life, not just fantasy!"
4,s5,TV Show,The Beatles: Get Back,NaN,"John Lennon, Paul McCartney, George Harrison, ...",NaN,"November 25, 2021",2021,NaN,1 Season,"Docuseries, Historical, Music",A three-part documentary from Peter Jackson ca...


In [53]:
df_hulu.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Ricky Velez: Here's Everything,NaN,NaN,NaN,"October 24, 2021",2021,TV-MA,NaN,"Comedy, Stand Up",​Comedian Ricky Velez bares it all with his ho...
1,s2,Movie,Silent Night,NaN,NaN,NaN,"October 23, 2021",2020,NaN,94 min,"Crime, Drama, Thriller","Mark, a low end South London hitman recently r..."
2,s3,Movie,The Marksman,NaN,NaN,NaN,"October 23, 2021",2021,PG-13,108 min,"Action, Thriller",A hardened Arizona rancher tries to protect an...
3,s4,Movie,Gaia,NaN,NaN,NaN,"October 22, 2021",2021,R,97 min,Horror,A forest ranger and two survivalists with a cu...
4,s5,Movie,Settlers,NaN,NaN,NaN,"October 22, 2021",2021,NaN,104 min,"Science Fiction, Thriller",Mankind's earliest settlers on the Martian fro...


In [55]:
df_netflix.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,None,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,None,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",None,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,None,None,None,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,None,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


* Shape de los datasets

In [54]:
#Shape de los datasets
for i,k in enumerate(dict_df):
    print(f'Forma de {k}: {dict_df[k].shape}')

Forma de df_amazon: (9668, 12)
Forma de df_disney: (1450, 12)
Forma de df_hulu: (3073, 12)
Forma de df_netflix: (8807, 12)


* Info de los datasets

In [45]:
df_amazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9668 entries, 0 to 9667
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       9668 non-null   object
 1   type          9668 non-null   object
 2   title         9668 non-null   object
 3   director      7586 non-null   object
 4   cast          8435 non-null   object
 5   country       672 non-null    object
 6   date_added    155 non-null    object
 7   release_year  9668 non-null   int64 
 8   rating        9331 non-null   object
 9   duration      9668 non-null   object
 10  listed_in     9668 non-null   object
 11  description   9668 non-null   object
dtypes: int64(1), object(11)
memory usage: 906.5+ KB


In [56]:
df_disney.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1450 entries, 0 to 1449
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       1450 non-null   object
 1   type          1450 non-null   object
 2   title         1450 non-null   object
 3   director      977 non-null    object
 4   cast          1260 non-null   object
 5   country       1231 non-null   object
 6   date_added    1447 non-null   object
 7   release_year  1450 non-null   int64 
 8   rating        1447 non-null   object
 9   duration      1450 non-null   object
 10  listed_in     1450 non-null   object
 11  description   1450 non-null   object
dtypes: int64(1), object(11)
memory usage: 136.1+ KB


In [57]:
df_hulu.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3073 entries, 0 to 3072
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   show_id       3073 non-null   object 
 1   type          3073 non-null   object 
 2   title         3073 non-null   object 
 3   director      3 non-null      object 
 4   cast          0 non-null      float64
 5   country       1620 non-null   object 
 6   date_added    3045 non-null   object 
 7   release_year  3073 non-null   int64  
 8   rating        2553 non-null   object 
 9   duration      2594 non-null   object 
 10  listed_in     3073 non-null   object 
 11  description   3069 non-null   object 
dtypes: float64(1), int64(1), object(10)
memory usage: 288.2+ KB


In [58]:
df_netflix.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 894.5+ KB


* Comprobacion de duplicados sin tener en cuenta la columna show_id

In [71]:
#Shape de los datasets
print('REGISTROS DUPLICADOS SIN CONSIDERAR LA COLUMNA show_id')
for i,k in enumerate(dict_df):
    print(f'''Valores duplicados {k}: {dict_df[k].drop(columns='show_id').duplicated().sum()}''')

REGISTROS DUPLICADOS SIN CONSIDERAR LA COLUMNA show_id
Valores duplicados df_amazon: 0
Valores duplicados df_disney: 0
Valores duplicados df_hulu: 0
Valores duplicados df_netflix: 0


In [73]:
print('REGISTROS DUPLICADOS SIN CONSIDERAR LA COLUMNA show_id')
for i,k in enumerate(dict_df):
    print(f'''Valores faltantes {k}: \n{dict_df[k].drop(columns='show_id').isna().sum()}\n''')

REGISTROS DUPLICADOS SIN CONSIDERAR LA COLUMNA show_id
Valores faltantes df_amazon: 
type               0
title              0
director        2082
cast            1233
country         8996
date_added      9513
release_year       0
rating           337
duration           0
listed_in          0
description        0
dtype: int64

Valores faltantes df_disney: 
type              0
title             0
director        473
cast            190
country         219
date_added        3
release_year      0
rating            3
duration          0
listed_in         0
description       0
dtype: int64

Valores faltantes df_hulu: 
type               0
title              0
director        3070
cast            3073
country         1453
date_added        28
release_year       0
rating           520
duration         479
listed_in          0
description        4
dtype: int64

Valores faltantes df_netflix: 
type               0
title              0
director        2634
cast             825
country          8

In [74]:
#Guardado del dataset de netflix a csv
df_netflix.to_csv('../Data/netflix_titles.csv', index=False)

* **Observaciones:**
    * Forma: Las columnas de los 4 datasets son las mismas, 12 en total, variando la cantidad de registros
    * Tipos de datos: Solo la columna _released_year_ es un tipo de dato entero en todas las columnas, el resto son de tipo string. La columna cast en df_hulu son todos valores tipo NaN por lo que es de tipo float
    * Valores unicos: No existen registros duplicados, todos son unicos
    * Valores faltantes:  Principalmente existen valores faltantes en las columnas _director_, _cast_, _country_, _date_added_, _rating_, _duration_, y poca cantidad de nulos en _listed_in_ y _description_  
    * Registros: Se observan algunas incongruencias en _rating_ que contiene la duración de peliculas o tv shows. Por lo que se debería pasar a la columna _duration_. Luego se deberian corregir algunos tipos de datos de algunas columnas de acuerdo a las necesidades. 
     
**Se decide cargar los archivos `.csv` a MySQL para realizar alguna limpieza y modificacion de los tipos que se consideren necesarias y coherentes a lo que se necesite. Previamente se pasa el dataset de netflix de json a csv para que se pueda cargar como se ve en la línea anterior**

### 3) Ingesta de datos a MySQL
* Se crea base de datos `moviesdb` en MySQL y las respectivas tablas con `LOAD LOCAL INFILE` y tratando de optimizar los tipos de datos
* A continuación de muestra un ejemplo de la tabla de movies_amazon y su posterior ingesta. De la misma manera se realiza para las otras tablas según el archivo [Carga e ingestión de datos](./ingestion_datos.sql)

```sql
      /*Creacion del Database*/
      DROP DATABASE moviesDB;
      CREATE DATABASE  IF NOT EXISTS `moviesDB`;
      USE `moviesDB`;

      /*Importacion de las tablas*/
      /*AMAZON*/
      DROP TABLE IF EXISTS `movies_amazon`;
      CREATE TABLE IF NOT EXISTS `movies_amazon` (
            `IdMovie` 		VARCHAR(5),
            `Type` 		VARCHAR(50),
            `Title` 		VARCHAR(150),
            `Director`		TEXT,
            `Cast` 		TEXT,
            `Country`		VARCHAR(200),
            `Date_Added`	VARCHAR(50),
            `Release_Year`	INT,
            `Rating`		VARCHAR(50),
            `Duration`		VARCHAR(50),
            `Listed_In`		VARCHAR(150),
            `Description`	TEXT
      ) ENGINE=InnoDB DEFAULT CHARSET=utf8mb4 COLLATE=utf8mb4_spanish_ci;
      /*Ingesta de datos*/
      LOAD DATA INFILE 'C:\\ProgramData\\MySQL\\MySQL Server 8.0\\Uploads\\amazon_prime_titles.csv'
      INTO TABLE `movies_amazon` 
      FIELDS TERMINATED BY ',' ENCLOSED BY '\"' ESCAPED BY ''
      LINES TERMINATED BY '\r\n' IGNORE 1 LINES;
```

### 3) Limpieza transformación de Datos
* A continuación se explican los pasos de limpieza realizados. El archivo sql se encuentra en [Limpieza de datos](./limpieza_transformacion.sql)  
**Nota: Se realiza una limpieza mínima pero suficiente según los requerimientos del proyecto**

##### Pasos

* 1- Se crea la tabla previamente agregando un IdMovie unico y autoincremental, también se agrega un campo Plataforma VARCHAR(10) que contiene el nombre de la plataforma a la quer pertenece. Se ingestan los datos con un SELECT y UNION de todos los campos de las tablas de cada plataforma excepto el campo `show_id`. También se agrega la plataforma correspondiente como sigue:

```sql
#Injesta de Peliculas de todas las plataformas en la tabla movies
INSERT INTO movies (Type, Title, Director, Cast, Country, Date_Added, Release_Year, Rating, Duration, Listed_In, Description, Plataforma)
SELECT Type, Title, Director, Cast, Country, Date_Added, Release_Year, Rating, Duration, Listed_In, Description, 'Amazon' FROM movies_amazon
UNION
SELECT Type, Title, Director, Cast, Country, Date_Added, Release_Year, Rating, Duration, Listed_In, Description, 'Disney' FROM movies_disney
UNION
SELECT Type, Title, Director, Cast, Country, Date_Added, Release_Year, Rating, Duration, Listed_In, Description, 'Hulu' FROM movies_hulu
UNION
SELECT Type, Title, Director, Cast, Country, Date_Added, Release_Year, Rating, Duration, Listed_In, Description, 'Netflix' FROM movies_netflix;
````

* 1- Se hace una limpieza de la columna Duration que es relevante para las consultas del proyecto realizando los siguientes pasos
    * Se crea columna copia de Duration para no perder los datos originales. Se buscan los datos de duración que se encuentran en Rating y se reemplazan en Duration donde hay vacíos
    * Se reemplazan los strings **Season**, **s** y **min**. Luego se hace un Trim para eliminar los espacios vacios y dejar solo el numero de duración
    * Se cambia el tipo de dato a `int` en la columna _Duration_ y se dropea _Duration_copy_

```sql
#Se agrega una columna de duración auxiliar llamada Duration_copy para no perder los datos y hacer las transformaciones
ALTER TABLE movies
ADD COLUMN Duration_copy VARCHAR(50) AFTER Duration;

#Se buscan los datos de duración en rating y se lo agrega en Duration_copy
UPDATE movies SET Duration_copy = Duration;
UPDATE movies SET Duration = Rating WHERE Rating LIKE '%min%' OR Rating LIKE '%Season%';

#Reemplazar min, Season y Seasons por NULL
UPDATE movies SET Duration = REPLACE(Duration,'Season','');
UPDATE movies SET Duration = REPLACE(Duration,'s','');
UPDATE movies SET Duration = TRIM(REPLACE(Duration,'min',''));
UPDATE movies SET Duration = REPLACE(Duration,'',NULL) WHERE Duration = '';

#Se cambia a INT los datos de duracion y dropeo la columna Duration_copy
ALTER TABLE movies
CHANGE COLUMN Duration Duration INT DEFAULT NULL,
DROP COLUMN Duration_copy;
```

### 4) Consultas pertinentes de prueba
* Se realizan algunas consultas según las responses que debe devolver la API para corroborar que funcionan correctamente luego.

```sql
#CONSULTA 1:  Máxima duracion
SELECT Title
FROM movies
WHERE Plataforma = 'Hulu' AND Release_Year = 2018
ORDER BY Duration DESC
LIMIT 1;

#CONSULTA 2: Cantidad de peliculas o tv shows
SELECT COUNT(*)
FROM(SELECT *
FROM movies
WHERE Type = 'Movie' AND Plataforma = 'Netflix') AS subquery;

#CONSULTA 3:  Cantidad de repeticion de un género en una plataforma
SELECT COUNT(*)
FROM(SELECT *
FROM movies
where Listed_In LIKE '%ComEdy%'and Plataforma = 'Amazon') AS subquery;
```

### 5) Conexión a MySQL, extracción y guardado de la tabla movies
* Se hace la conexión via `pymysql` para obtener el dataframe de movies final y guardarlo en la carpeta [Data](../Data/) para la consulta en los decoradores con FastAPI

In [3]:
#CONEXION
conexion = pymysql.connect(
    host = 'localhost',
    user = 'root',
    passwd = 'lazarte5',
    database = 'moviesdb'
)
#CURSOR
cursor = conexion.cursor()

In [4]:
#Extraxión la tabla movies y guardado en df_movies a partir del método de pandas read_sql_query
df_movies = pd.read_sql_query('SELECT * FROM movies', conexion)

C:\Users\Nico\AppData\Local\Temp\ipykernel_5368\3624247885.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_movies = pd.read_sql_query('SELECT * FROM movies', conexion)


In [5]:
#CIERRE DE LA CONEXION
conexion.close()

In [6]:
#Se guarda df_movies en un archivo csv
df_movies.to_csv('../Data/movies_titles.csv', index=False)

In [7]:
df_movies.shape

(22998, 13)

In [12]:
df_movies= pd.read_csv('../Data/movies_titles.csv')

In [11]:
#Verificación de que se guardó el archivo correctamente
df_movies.shape

(22998, 13)